<a href="https://colab.research.google.com/github/Bharathikoruprolu245/EodPaymentSystem-/blob/main/batch/AlphaFold2_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ColabFold v1.5.5: AlphaFold2 w/ MMseqs2 BATCH

<img src="https://raw.githubusercontent.com/sokrypton/ColabFold/main/.github/ColabFold_Marv_Logo_Small.png" height="256" align="right" style="height:256px">

Easy to use AlphaFold2 protein structure [(Jumper et al. 2021)](https://www.nature.com/articles/s41586-021-03819-2) and complex [(Evans et al. 2021)](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v1) prediction using multiple sequence alignments generated through MMseqs2. For details, refer to our manuscript:

[Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S, Steinegger M. ColabFold: Making protein folding accessible to all.
*Nature Methods*, 2022](https://www.nature.com/articles/s41592-022-01488-1)

**Usage**

`input_dir` directory with only fasta files or MSAs stored in Google Drive. MSAs need to be A3M formatted and have an `.a3m` extention. For MSAs MMseqs2 will not be called.

`result_dir` results will be written to the result directory in Google Drive

Old versions: [v1.4](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.4.0/batch/AlphaFold2_batch.ipynb), [v1.5.1](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.5.1/batch/AlphaFold2_batch.ipynb), [v1.5.2](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.5.2/batch/AlphaFold2_batch.ipynb), [v1.5.3-patch](https://colab.research.google.com/github/sokrypton/ColabFold/blob/56c72044c7d51a311ca99b953a71e552fdc042e1/batch/AlphaFold2_batch.ipynb)

<strong>For more details, see <a href="#Instructions">bottom</a> of the notebook and checkout the [ColabFold GitHub](https://github.com/sokrypton/ColabFold). </strong>

-----------

### News
- <b><font color='green'>2023/07/31: The ColabFold MSA server is back to normal. It was using older DB (UniRef30 2202/PDB70 220313) from 27th ~8:30 AM CEST to 31st ~11:10 AM CEST.</font></b>
- <b><font color='green'>2023/06/12: New databases! UniRef30 updated to 2023_02 and PDB to 230517. We now use PDB100 instead of PDB70 (see notes in the [main](https://colabfold.com) notebook).</font></b>
- <b><font color='green'>2023/06/12: We introduced a new default pairing strategy: Previously, for multimer predictions with more than 2 chains, we only pair if all sequences taxonomically match ("complete" pairing). The new default "greedy" strategy pairs any taxonomically matching subsets.</font></b>

In [1]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

Mounted at /content/drive


In [2]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`

input_dir = '/content/drive/MyDrive/individual_fastas/batch2' #@param {type:"string"}
result_dir = '/content/drive/MyDrive/result2' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 3 #@param [1,2,3,4,5] {type:"raw"}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
stop_at_score = 100 #@param {type:"string"}
#@markdown - early stop computing models once score > threshold (avg. plddt for "structures" and ptmscore for "complexes")
use_custom_msa = False
num_relax = 0 #@param [0, 1, 5] {type:"raw"}
use_amber = num_relax > 0
relax_max_iterations = 0 #@param [0,200,2000] {type:"raw"}
use_templates = True #@param {type:"boolean"}
do_not_overwrite_results = True #@param {type:"boolean"}
zip_results = True #@param {type:"boolean"}


In [3]:
!pip install appdirs biopython tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 103.2 MB/s eta 0:00:00


In [4]:
#@title Install dependencies
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  # install ColabFold
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"

  if [ -n "${TPU_NAME}" ]; then
    pip install -q --no-warn-conflicts -U dm-haiku==0.0.10 jax==0.3.25
  fi

  ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
  ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold

  # hack to fix TF crash
  rm -f /usr/local/lib/python3.*/dist-packages/tensorflow/core/kernels/libtfkernel_sobol_op.so

  touch COLABFOLD_READY
fi

# Download params (~1min)
python3 -m colabfold.download

# setup conda if needed
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-25.3.1-0-Linux-x86_64.sh
    bash Miniforge3-25.3.1-0-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniforge3-25.3.1-0-Linux-x86_64.sh
    conda config --set auto_update_conda false
    touch CONDA_READY
  fi
fi

# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi

# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=8.2.0 python="${PYTHON_VERSION}" pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 7.9 MB/s eta 0:00:00
warning  libmamba [python-3.12.11-h9e4cc4f_0_cpython] The following files were already present in the environment:
    - bin/python
warning  libmamba [charset-normalizer-3.4.2-pyhd8ed1ab_0] The following files were already present in the environment:
    - bin/normalizer
warning  libmamba [distro-1.9.0-pyhd8ed1ab_1] The following files were already present in the environment:
    - bin/distro
warning  libmamba [jsonpointer-3.0.0-py312h7900ff3_1] The following files were already present in the environment:
    - bin/jsonpointer
warning  libmamba [wheel-0.45.1-pyhd8ed1ab_1] The following files were

In [5]:
#@title Run Prediction

import sys

from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path

# For some reason we need that to get pdbfixer to import
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

setup_logging(Path(result_dir).joinpath("log.txt"))

queries, is_complex = get_queries(input_dir)
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    num_relax=num_relax,
    relax_max_iterations=relax_max_iterations,
    msa_mode=msa_mode,
    model_type="auto",
    num_models=num_models,
    num_recycles=num_recycles,
    model_order=[1, 2, 3, 4, 5],
    is_complex=is_complex,
    data_dir=default_data_dir,
    keep_existing_results=do_not_overwrite_results,
    rank_by="auto",
    pair_mode="unpaired+paired",
    stop_at_score=stop_at_score,
    zip_results=zip_results,
    user_agent="colabfold/google-colab-batch",
)

2025-08-17 12:13:43,042 Running on GPU
2025-08-17 12:13:43,401 Found 8 citations for tools or databases
2025-08-17 12:13:43,402 Query 1/9: pilin_WP_011051752.1 (length 143)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:03 remaining: 00:00]


2025-08-17 12:13:53,659 Sequence 0 found templates: ['6vk9_A', '2m7g_A', '5vxx_A', '2pil_A', '2hi2_A']
2025-08-17 12:14:16,942 Padding length to 153
2025-08-17 12:15:03,266 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=91.2 pTM=0.753
2025-08-17 12:15:43,709 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=91.4 pTM=0.761 tol=0.236
2025-08-17 12:15:55,319 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=91.6 pTM=0.758 tol=0.238
2025-08-17 12:16:07,098 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=91.5 pTM=0.756 tol=0.0913
2025-08-17 12:16:07,099 alphafold2_ptm_model_1_seed_000 took 110.2s (3 recycles)
2025-08-17 12:16:18,974 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=91 pTM=0.756
2025-08-17 12:16:30,692 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=91.1 pTM=0.762 tol=0.364
2025-08-17 12:16:42,378 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=91.1 pTM=0.761 tol=0.133
2025-08-17 12:16:53,988 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=91.2 pTM=0.765 tol=0.141
2025-08-17 12:1

COMPLETE: 100%|██████████| 150/150 [elapsed: 00:02 remaining: 00:00]


2025-08-17 12:18:41,634 Sequence 0 found templates: ['4b5c_C', '4g4x_A', '2hqs_H', '2lca_A', '4pwt_A', '4pwt_A', '5lkw_A', '2lbt_A', '2l26_A', '2k1s_A', '6jfw_A', '5u1h_A', '5wtl_C', '5eaz_A', '6iki_A', '5wtp_B', '4rha_A', '2mqe_A', '7rjj_A']
2025-08-17 12:18:51,128 Padding length to 182
2025-08-17 12:19:41,977 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=83.6 pTM=0.688
2025-08-17 12:20:27,149 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=83.7 pTM=0.691 tol=1.95
2025-08-17 12:20:41,474 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=83.7 pTM=0.689 tol=1.37
2025-08-17 12:20:55,657 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=84 pTM=0.694 tol=1.46
2025-08-17 12:20:55,658 alphafold2_ptm_model_1_seed_000 took 124.5s (3 recycles)
2025-08-17 12:21:09,697 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=84.1 pTM=0.692
2025-08-17 12:21:23,588 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=84.1 pTM=0.696 tol=2.19
2025-08-17 12:21:37,491 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=84

...

RUNNING:  50%|█████     | 75/150 [elapsed: 03:36 remaining: 01:40]

2025-08-17 13:33:30,913 Sleeping for 8s. Reason: RUNNING


COMPLETE: 100%|██████████| 150/150 [elapsed: 03:47 remaining: 00:00]


2025-08-17 13:33:49,559 Sequence 0 found templates: ['4avr_B', '5ntb_B', '6i05_A', '4jjo_A', '1x60_A', '4fer_A', '7wvr_A', '1x60_A', '6i05_A', '4avr_B']
2025-08-17 13:35:26,291 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=68.6 pTM=0.474
2025-08-17 13:36:26,338 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=69.2 pTM=0.485 tol=11.7
2025-08-17 13:37:26,542 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=69.2 pTM=0.479 tol=5.1
2025-08-17 13:38:26,527 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=69.6 pTM=0.481 tol=3.98
2025-08-17 13:38:26,528 alphafold2_ptm_model_1_seed_000 took 240.6s (3 recycles)
2025-08-17 13:39:27,021 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=66.6 pTM=0.458
2025-08-17 13:40:27,345 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=67.7 pTM=0.465 tol=11.3
2025-08-17 13:41:27,520 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=67.6 pTM=0.466 tol=6.28
2025-08-17 13:42:28,031 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=68.4 pTM=0.469 tol=2.92
2025-08-17 13:42:

{'rank': [['rank_001_alphafold2_ptm_model_3_seed_000',
   'rank_002_alphafold2_ptm_model_1_seed_000',
   'rank_003_alphafold2_ptm_model_2_seed_000'],
  ['rank_001_alphafold2_ptm_model_2_seed_000',
   'rank_002_alphafold2_ptm_model_1_seed_000',
   'rank_003_alphafold2_ptm_model_3_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000',
   'rank_002_alphafold2_ptm_model_3_seed_000',
   'rank_003_alphafold2_ptm_model_2_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000',
   'rank_002_alphafold2_ptm_model_3_seed_000',
   'rank_003_alphafold2_ptm_model_2_seed_000'],
  ['rank_001_alphafold2_ptm_model_3_seed_000',
   'rank_002_alphafold2_ptm_model_1_seed_000',
   'rank_003_alphafold2_ptm_model_2_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000',
   'rank_002_alphafold2_ptm_model_2_seed_000',
   'rank_003_alphafold2_ptm_model_3_seed_000'],
  ['rank_001_alphafold2_ptm_model_3_seed_000',
   'rank_002_alphafold2_ptm_model_2_seed_000',
   'rank_003_alphafold2_ptm_model_1_seed_000']

# Instructions <a name="Instructions"></a>
**Quick start**
1. Upload your single fasta files to a folder in your Google Drive
2. Define path to the fold containing the fasta files (`input_dir`) define an outdir (`output_dir`)
3. Press "Runtime" -> "Run all".

**Result zip file contents**

At the end of the job a all results `jobname.result.zip` will be uploaded to your (`output_dir`) Google Drive. Each zip contains one protein.

1. PDB formatted structures sorted by avg. pIDDT. (unrelaxed and relaxed if `use_amber` is enabled).
2. Plots of the model quality.
3. Plots of the MSA coverage.
4. Parameter log file.
5. A3M formatted input MSA.
6. BibTeX file with citations for all used tools and databases.


**Troubleshooting**
* Check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".
* Try to restart the session "Runtime" -> "Factory reset runtime".
* Check your input sequence.

**Known issues**
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Your browser can block the pop-up for downloading the result file. You can choose the `save_to_google_drive` option to upload to Google Drive instead or manually download the result file: Click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

**Limitations**
* Computing resources: Our MMseqs2 API can handle ~20-50k requests per day.
* MSAs: MMseqs2 is very precise and sensitive but might find less hits compared to HHblits/HMMer searched against BFD or Mgnify.
* We recommend to additionally use the full [AlphaFold2 pipeline](https://github.com/deepmind/alphafold).

**Description of the plots**
*   **Number of sequences per position** - We want to see at least 30 sequences per position, for best performance, ideally 100 sequences.
*   **Predicted lDDT per position** - model confidence (out of 100) at each position. The higher the better.
*   **Predicted Alignment Error** - For homooligomers, this could be a useful metric to assess how confident the model is about the interface. The lower the better.

**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/sokrypton/ColabFold/issues

**License**

The source code of ColabFold is licensed under [MIT](https://raw.githubusercontent.com/sokrypton/ColabFold/main/LICENSE). Additionally, this notebook uses AlphaFold2 source code and its parameters licensed under [Apache 2.0](https://raw.githubusercontent.com/deepmind/alphafold/main/LICENSE) and  [CC BY 4.0](https://creativecommons.org/licenses/by-sa/4.0/) respectively. Read more about the AlphaFold license [here](https://github.com/deepmind/alphafold).

**Acknowledgments**
- We thank the AlphaFold team for developing an excellent model and open sourcing the software.

- Do-Yoon Kim for creating the ColabFold logo.

- A colab by Sergey Ovchinnikov ([@sokrypton](https://twitter.com/sokrypton)), Milot Mirdita ([@milot_mirdita](https://twitter.com/milot_mirdita)) and Martin Steinegger ([@thesteinegger](https://twitter.com/thesteinegger)).
